In [4]:
import xml.etree.ElementTree as ET
import random

# Parsing XML files
def parse_calls(file_path):
    calls = []
    tree = ET.parse(file_path)
    root = tree.getroot()
    for call in root.findall("call"):
        title = call.get("title")
        link = call.get("link")
        calls.append({"title": title, "link": link})
    return calls

def parse_formations(file_path):
    formations = {}
    tree = ET.parse(file_path)
    root = tree.getroot()
    for formation in root.findall("formation"):
        name = formation.get("name")
        dancers = []
        for dancer in formation.findall("dancer"):
            dancers.append({
                "gender": dancer.get("gender"),
                "x": float(dancer.get("x")),
                "y": float(dancer.get("y")),
                "angle": float(dancer.get("angle"))
            })
        formations[name] = dancers
    return formations

def parse_moves(file_path):
    moves = {}
    tree = ET.parse(file_path)
    root = tree.getroot()
    for path in root.findall("path"):
        name = path.get("name")
        movements = []
        for movement in path.findall("movement"):
            movements.append({
                "hands": movement.get("hands"),
                "beats": float(movement.get("beats")),
                "cx1": float(movement.get("cx1")),
                "cy1": float(movement.get("cy1")),
                "cx2": float(movement.get("cx2")),
                "cy2": float(movement.get("cy2")),
                "x2": float(movement.get("x2")),
                "y2": float(movement.get("y2"))
            })
        moves[name] = movements
    return moves

# Loading data
calls = parse_calls("../assets/src/calls.xml")
formations = parse_formations("../assets/src/formations.xml")
moves = parse_moves("../assets/src/moves.xml")

import random

class DancePatternGenerator:
    def __init__(self, calls, formations, moves):
        self.calls = calls
        self.formations = formations
        self.moves = moves

    def generate_sequence(self, start_formation, max_calls=10):
        """
        Generates a randomized sequence of dance calls up to a specified maximum, following formation rules.
        
        Parameters:
        start_formation (str): The initial formation to start the sequence from.
        max_calls (int): Maximum number of calls to include in the sequence.

        Returns:
        list: A sequence of call titles.
        """
        sequence = []
        current_formation = start_formation

        for _ in range(max_calls):
            # Filter valid calls based on the current formation
            valid_calls = [call for call in self.calls if self.is_valid_transition(call, current_formation)]
            
            if not valid_calls:
                # If no valid calls are found, end the sequence
                break
            
            # Randomly select a valid call
            selected_call = random.choice(valid_calls)
            sequence.append(selected_call["title"])
            
            # Update the formation after the selected call
            current_formation = self.update_formation(selected_call, current_formation)

        return sequence

    def is_valid_transition(self, call, current_formation):
        """
        Checks if a call can follow the current formation.
        
        Parameters:
        call (dict): The call to check.
        current_formation (str): The current formation.

        Returns:
        bool: True if the call is valid, otherwise False.
        """
        # Placeholder: Implement logic to check if the call is valid based on current_formation
        # For now, assume all calls are valid
        return True

    def update_formation(self, selected_call, current_formation):
        """
        Updates the formation after a call is made.
        
        Parameters:
        selected_call (dict): The call that was selected.
        current_formation (str): The current formation.

        Returns:
        str: The updated formation after the call.
        """
        # Placeholder: Implement logic to update formation based on selected call
        # For now, keep the same formation
        return current_formation


# Initialize generator with loaded data
generator = DancePatternGenerator(calls, formations, moves)

# Generate a sequence with random calls, ensuring a maximum of 5 calls in the sequence
generated_sequence = generator.generate_sequence("Facing Couples", max_calls=5)
print("Generated Dance Sequence:")
print(generated_sequence)


Generated Dance Sequence:
['Interlocked Ramble', 'Star Thru', 'Linear Action', 'Switch the Wave', 'Quarter In']


Patter Generator V1 Includes:

- Starter functions to parse XML files for calls, formations, and moves.
- A DancePatternGenerator class to generate a random sequence of dance calls.
- Introduces methods for checking valid transitions and updating formations (currently placeholders).
- Loads data from XML files located in the project assets.
- Generates and prints a sample dance sequence with a variable of 5 calls.